In [ ]:
import random
import string
from io import BytesIO

from base64 import b64encode
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
matplotlib.use('Agg')
import cherrypy
from app.htmlx import Html


class TemplateHtml(Html):
    def __init__(self, template_path, config_path):
        super().__init__(template_path, config_path)
        
    @cherrypy.expose
    def display2(self):
        # Modified from https://gist.github.com/tkf/1299670 by @tkf
        d = self.plot()
        d = d.decode('utf8')
        return f'''<img src="data:image/png;base64,{d}" width="640" height="480" border="0">'''

    def plot(self):
        image = BytesIO()
        x = np.linspace(0, 10)
        y = np.sin(x)
        plt.plot(x, y)
        plt.savefig(image, format='png')
        return b64encode(image.getvalue())

    @cherrypy.expose
    def index(self):
        yield self.encode('utf-8')

    @cherrypy.expose
    def generate(self, length=8):
        some_string = ''.join(random.sample(string.hexdigits, int(length)))
        cherrypy.session['mystring'] = some_string
        yield some_string

    @cherrypy.expose
    def display(self):
        yield cherrypy.session['mystring']

    def quickstart(self, script_name='', config=None):
        cherrypy.quickstart(self, script_name, config)


template_path = 'templates/editor.html'
config_path = 'configs/editor.toml'
config = 'configs/base-server.conf'
httpd = TemplateHtml(template_path, config_path)
httpd.quickstart(config=config)